In [1]:
from utils.BackTest import Engine, Strategy
import pandas as pd

In [2]:
df = pd.read_csv("../data/processed/features_trading_strategy.csv")

In [3]:
df = df.rename(columns={'OPEN':'Open','HIGH':'High','LOW':'Low','CLOSE':'Close','VOLUME':'Volume'})

In [4]:
df['Close']

0        25766.93
1        25869.97
2        26159.98
3        26098.99
4        26121.84
           ...   
13495    86974.07
13496    86835.24
13497    86913.05
13498    87206.52
13499    87060.41
Name: Close, Length: 13500, dtype: float64

In [5]:
class SMACrossOverBuySideWithTPSL(Strategy):
    def __init__(self):
        super().__init__()
        # Default multipliers for TP and SL calculations
        self.tp_atr_multiplier = 1.5   
        self.sl_atr_multiplier = 1.5  
    def on_bar(self):
        # Get current bar data
        current_bar = self.data.loc[self.current_idx]
        close = current_bar['Close']
        sma20 = current_bar['SMA20']
        sma50 = current_bar['SMA50']

        # For multiple positions, get all BTC positions
        btc_positions = [p for p in self.positions if p.ticker == 'BTC']

        # Buy signal: SMA20 > SMA50, and we allow up to 5 positions.
        if sma20 > sma50:
            if len(btc_positions) < 5:
                # Use 10% of available cash to determine the order size.
                order_size = 0.1 * self.cash / close
                self.buy('BTC', order_size)
                print(f"{self.current_idx} BUY signal: entering new position, order size {order_size:.4f}")
            else:
                print(f"{self.current_idx} BUY signal: maximum BTC positions reached.")


In [6]:
class SMACrossOver(Strategy):
    """
    Incrementally adjusts the net position based on SMA50 vs. SMA200:
      - If SMA50 > SMA200: Buy 1 unit (up to a max of +5) each bar.
      - If SMA50 < SMA200: Sell 1 unit (down to a max of -5) each bar.
      - If SMA50 == SMA200: Do nothing.
    """

    def on_bar(self):
        # Get current bar data
        current_bar = self.data.loc[self.current_idx]
        close_price = current_bar['Close']
        sma50 = current_bar['SMA_50']
        sma200 = current_bar['SMA_200']

        # Check current net position for BTC
        btc_position = self.get_position('BTC')
        current_size = btc_position.size if btc_position else 0

        # Compare SMAs to decide incremental move
        if sma50 > sma200:
            # Move net position +1 toward max +5
            if current_size < 5:
                self.buy('BTC', size=1)
                print(f"{self.current_idx} Incremental BUY: size -> {current_size + 1}")
        elif sma50 < sma200:
            # Move net position -1 toward min -5
            if current_size > -5:
                self.sell('BTC', size=1)
                print(f"{self.current_idx} Incremental SELL: size -> {current_size - 1}")
        else:
            # sma50 == sma200, do nothing
            print(f"{self.current_idx} SMAs are equal, no change.")


In [6]:
df['SMA50'] = df['Close'].rolling(window=50).mean()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13500 entries, 0 to 13499
Data columns (total 45 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   datetime            13500 non-null  object 
 1   Open                13500 non-null  float64
 2   High                13500 non-null  float64
 3   Low                 13500 non-null  float64
 4   Close               13500 non-null  float64
 5   TOTAL_TRADES        13500 non-null  int64  
 6   TOTAL_TRADES_BUY    13500 non-null  int64  
 7   TOTAL_TRADES_SELL   13500 non-null  int64  
 8   Volume              13500 non-null  float64
 9   VOLUME_BUY          13500 non-null  float64
 10  VOLUME_SELL         13500 non-null  float64
 11  return              13499 non-null  float64
 12  log_return          13499 non-null  float64
 13  hourly_volatility   13488 non-null  float64
 14  EMA_8               13500 non-null  float64
 15  EMA_13              13500 non-null  float64
 16  EMA_

In [8]:
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.set_index('datetime')

In [9]:
df_filtered = df.dropna()
df_filtered

,Open,High,Low,Close,TOTAL_TRADES,TOTAL_TRADES_BUY,TOTAL_TRADES_SELL,Volume,VOLUME_BUY,VOLUME_SELL,...,%D,Stochastic_Signal,OBV,OBV_EMA20,OBV_Signal,H-L,H-PC,L-PC,TR,ATR
datetime,,,,,,,,,,,,,,,,,,,,,
2023-09-20 15:00:00+00:00,27124.99,27199.00,27101.38,27180.63,28653,13370,15283,1142.66083,546.71174,595.94909,...,56.176984,0,1142.66083,-14.030193,1,97.62,74.01,23.61,97.62,131.745000
2023-09-20 16:00:00+00:00,27180.63,27225.00,27140.05,27161.15,28758,13284,15474,1046.99060,514.99520,531.99540,...,69.913266,0,-1046.99060,-112.407375,-1,84.95,44.37,40.58,84.95,124.016667
2023-09-20 17:00:00+00:00,27161.15,27277.36,27155.64,27195.12,35378,17481,17897,1316.64158,662.48215,654.15943,...,92.855977,0,1316.64158,23.692526,1,121.72,116.21,5.51,121.72,134.663333
2023-09-20 18:00:00+00:00,27195.12,27300.00,27079.38,27165.00,71225,36043,35182,2906.08791,1454.70556,1451.38235,...,88.539328,-1,-2906.08791,-255.334182,-1,220.62,104.88,115.74,220.62,149.423333
2023-09-20 19:00:00+00:00,27165.00,27187.19,26800.00,26930.61,74847,32923,41924,3708.89251,1572.21211,2136.68040,...,60.764950,0,-3708.89251,-584.244499,0,387.19,22.19,365.00,387.19,183.123333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-27 15:00:00+00:00,87297.33,87575.00,86752.58,86974.07,180420,91975,88445,1554.66361,699.72490,854.93871,...,38.966585,0,-1554.66361,-216.223678,-1,822.42,277.67,544.75,822.42,736.608333
2025-03-27 16:00:00+00:00,86974.07,87271.43,86805.20,86835.24,129979,66364,63615,671.69986,244.03675,427.66311,...,50.664024,0,-671.69986,-259.602362,0,466.23,297.36,168.87,466.23,732.878333
2025-03-27 17:00:00+00:00,86835.24,87150.95,86796.00,86913.05,120897,50367,70530,596.49192,257.41179,339.08013,...,42.496856,0,596.49192,-178.069573,1,354.95,315.71,39.24,354.95,703.821667


In [10]:
e = Engine()
e.add_data(df_filtered)
e.add_strategy(SMACrossOver())
stats = e.run()     

  9%|▊         | 1152/13301 [00:00<00:02, 5843.15it/s]

2023-09-20 15:00:00+00:00 Incremental BUY: size -> 1
2023-09-20 16:00:00+00:00 Incremental BUY: size -> 2
2023-09-20 17:00:00+00:00 Incremental BUY: size -> 3
2023-09-20 18:00:00+00:00 Incremental BUY: size -> 4
2023-09-20 19:00:00+00:00 Not enough cash to buy additional size.
2023-09-20 19:00:00+00:00 Incremental BUY: size -> 4
2023-09-20 20:00:00+00:00 Not enough cash to buy additional size.
2023-09-20 20:00:00+00:00 Incremental BUY: size -> 4
2023-09-20 21:00:00+00:00 Not enough cash to buy additional size.
2023-09-20 21:00:00+00:00 Incremental BUY: size -> 4
2023-09-20 22:00:00+00:00 Not enough cash to buy additional size.
2023-09-20 22:00:00+00:00 Incremental BUY: size -> 4
2023-09-20 23:00:00+00:00 Not enough cash to buy additional size.
2023-09-20 23:00:00+00:00 Incremental BUY: size -> 4
2023-09-21 00:00:00+00:00 Not enough cash to buy additional size.
2023-09-21 00:00:00+00:00 Incremental BUY: size -> 4
2023-09-21 01:00:00+00:00 Not enough cash to buy additional size.
2023-09-

 18%|█▊        | 2439/13301 [00:00<00:01, 6221.72it/s]

2023-11-22 10:00:00+00:00 Not enough cash to buy additional size.
2023-11-22 10:00:00+00:00 Incremental BUY: size -> 3
2023-11-22 11:00:00+00:00 Not enough cash to buy additional size.
2023-11-22 11:00:00+00:00 Incremental BUY: size -> 3
2023-11-22 12:00:00+00:00 Not enough cash to buy additional size.
2023-11-22 12:00:00+00:00 Incremental BUY: size -> 3
2023-11-22 13:00:00+00:00 Not enough cash to buy additional size.
2023-11-22 13:00:00+00:00 Incremental BUY: size -> 3
2023-11-22 14:00:00+00:00 Not enough cash to buy additional size.
2023-11-22 14:00:00+00:00 Incremental BUY: size -> 3
2023-11-22 15:00:00+00:00 Not enough cash to buy additional size.
2023-11-22 15:00:00+00:00 Incremental BUY: size -> 3
2023-11-22 16:00:00+00:00 Not enough cash to buy additional size.
2023-11-22 16:00:00+00:00 Incremental BUY: size -> 3
2023-11-22 17:00:00+00:00 Not enough cash to buy additional size.
2023-11-22 17:00:00+00:00 Incremental BUY: size -> 3
2023-11-22 18:00:00+00:00 Not enough cash to buy

 28%|██▊       | 3761/13301 [00:00<00:01, 5645.19it/s]

...

2024-10-19 20:00:00+00:00 Not enough cash to buy additional size.
2024-10-19 20:00:00+00:00 Incremental BUY: size -> 2
2024-10-19 21:00:00+00:00 Not enough cash to buy additional size.
2024-10-19 21:00:00+00:00 Incremental BUY: size -> 2
2024-10-19 22:00:00+00:00 Not enough cash to buy additional size.
2024-10-19 22:00:00+00:00 Incremental BUY: size -> 2
2024-10-19 23:00:00+00:00 Not enough cash to buy additional size.
2024-10-19 23:00:00+00:00 Incremental BUY: size -> 2
2024-10-20 00:00:00+00:00 Not enough cash to buy additional size.
2024-10-20 00:00:00+00:00 Incremental BUY: size -> 2
2024-10-20 01:00:00+00:00 Not enough cash to buy additional size.
2024-10-20 01:00:00+00:00 Incremental BUY: size -> 2
2024-10-20 02:00:00+00:00 Not enough cash to buy additional size.
2024-10-20 02:00:00+00:00 Incremental BUY: size -> 2
2024-10-20 03:00:00+00:00 Not enough cash to buy additional size.
2024-10-20 03:00:00+00:00 Incremental BUY: size -> 2
2024-10-20 04:00:00+00:00 Not enough cash to buy

 95%|█████████▌| 12662/13301 [00:02<00:00, 5723.72it/s]

2025-01-06 11:00:00+00:00 Not enough cash to buy additional size.
2025-01-06 11:00:00+00:00 Incremental BUY: size -> 1
2025-01-06 12:00:00+00:00 Not enough cash to buy additional size.
2025-01-06 12:00:00+00:00 Incremental BUY: size -> 1
2025-01-06 13:00:00+00:00 Not enough cash to buy additional size.
2025-01-06 13:00:00+00:00 Incremental BUY: size -> 1
2025-01-06 14:00:00+00:00 Not enough cash to buy additional size.
2025-01-06 14:00:00+00:00 Incremental BUY: size -> 1
2025-01-06 15:00:00+00:00 Not enough cash to buy additional size.
2025-01-06 15:00:00+00:00 Incremental BUY: size -> 1
2025-01-06 16:00:00+00:00 Not enough cash to buy additional size.
2025-01-06 16:00:00+00:00 Incremental BUY: size -> 1
2025-01-06 17:00:00+00:00 Not enough cash to buy additional size.
2025-01-06 17:00:00+00:00 Incremental BUY: size -> 1
2025-01-06 18:00:00+00:00 Not enough cash to buy additional size.
2025-01-06 18:00:00+00:00 Incremental BUY: size -> 1
2025-01-06 19:00:00+00:00 Not enough cash to buy

100%|██████████| 13301/13301 [00:02<00:00, 5937.14it/s]

2025-03-16 18:00:00+00:00 Not enough cash to buy additional size.
2025-03-16 18:00:00+00:00 Incremental BUY: size -> 1
2025-03-16 19:00:00+00:00 Not enough cash to buy additional size.
2025-03-16 19:00:00+00:00 Incremental BUY: size -> 1
2025-03-16 20:00:00+00:00 Not enough cash to buy additional size.
2025-03-16 20:00:00+00:00 Incremental BUY: size -> 1
2025-03-16 21:00:00+00:00 Not enough cash to buy additional size.
2025-03-16 21:00:00+00:00 Incremental BUY: size -> 1
2025-03-16 22:00:00+00:00 Not enough cash to buy additional size.
2025-03-16 22:00:00+00:00 Incremental BUY: size -> 1
2025-03-16 23:00:00+00:00 Not enough cash to buy additional size.
2025-03-16 23:00:00+00:00 Incremental BUY: size -> 1
2025-03-17 00:00:00+00:00 Not enough cash to buy additional size.
2025-03-17 00:00:00+00:00 Incremental BUY: size -> 1
2025-03-17 01:00:00+00:00 Not enough cash to buy additional size.
2025-03-17 01:00:00+00:00 Incremental BUY: size -> 1
2025-03-17 02:00:00+00:00 Not enough cash to buy

In [11]:
e.plot()